<a href="https://colab.research.google.com/github/JayanthK99/Jayanth_INFO5731_Spring2025/blob/main/Kaddipudi_Jayanth_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [9]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import csv

main_url = "https://ddr.densho.org/narrators/?page={}"

def extract_narrator_info():
    all_narrators_data = []

    for page_num in range(1, 42):
        link1 = Request(main_url.format(page_num), headers={'User-Agent': 'Mozilla/5.0'})
        url1 = urlopen(link1)
        data1 = url1.read()
        data1_soup = BeautifulSoup(data1, 'html.parser')

        for narrator_link in data1_soup.find_all('h4'):
            link2 = Request(narrator_link.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
            url2 = urlopen(link2)
            data2 = url2.read()
            data2_soup = BeautifulSoup(data2, 'html.parser')

            narrator = data2_soup.find("div", attrs={'class': 'col-sm-8 col-md-8'})
            narrator_name = narrator.h1.text.strip().replace('"', "")
            narrator_bio = narrator.p.text.strip() if narrator.p else ""

            interview_titles = []
            dates_locations = []
            densho_ids = []

            for interview in data2_soup.find_all("div", attrs={'class': 'media'}):
                interview_title = interview.find("b", attrs={'class': 'media-heading'}).text.strip()
                interview_details = interview.find("div", attrs={'class': 'source muted'})
                date_location = ""

                if interview_details:
                    details_text = interview_details.get_text("\n").strip().split("\n")
                    if len(details_text) >= 2:
                        date = details_text[0].strip()
                        location = details_text[1].strip()
                        date_location = f"{date}, {location}"

                interview_link = interview.find("a", href=True)
                densho_id = interview_link['href'].split('/')[-2] if interview_link else ""

                interview_titles.append(interview_title)
                dates_locations.append(date_location)
                densho_ids.append(densho_id)

            all_narrators_data.append({
                'Narrator Name': narrator_name,
                'Bio': narrator_bio,
                'Interview Title': ", ".join(interview_titles),
                'Date & Location': ", ".join(dates_locations),
                'Densho ID': ", ".join(densho_ids)
            })

    csv_file = "List_of_narrators.csv"
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=[
            'Narrator Name', 'Bio', 'Interview Title', 'Date & Location', 'Densho ID'
        ])
        writer.writeheader()
        writer.writerows(all_narrators_data)

    print(f"All narrators' data saved to '{csv_file}'.")

extract_narrator_info()


All narrators' data saved to 'List_of_narrators.csv'.


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [10]:
#(1) Remove noise, such as special characters and punctuations.
import pandas as pd
import re

file_path = "/content/List_of_narrators.csv"
output_file_path = "/content/Step_1_List_of_narrators.csv"

df = pd.read_csv(file_path)

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\|+', ' ', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
    return text
df['Bio'] = df['Bio'].astype(str).apply(clean_text)
df['Interview Title'] = df['Interview Title'].astype(str).apply(clean_text)
df['Date & Location'] = df['Date & Location'].astype(str).apply(clean_text)
df['Densho ID'] = df['Densho ID'].astype(str).apply(clean_text)
df.to_csv(output_file_path, index=False)
df_cleaned = pd.read_csv(output_file_path)
df_cleaned.head()

,Narrator Name,Bio,Interview Title,Date & Location,Densho ID
0,Kay Aiko Abe,Nisei female Born May 9 1927 in Selleck Washin...,Kay Aiko Abe Interview ddrdensho1000232,December 2 2008 Seattle Washington,ddrdensho1000232
1,Art Abe,Nisei male Born June 12 1921 in Seattle Washin...,Art Abe Interview ddrdensho1000206,January 24 2008 Seattle Washington,ddrdensho1000206
2,Sharon Tanagi Aburano,Nisei female Born October 31 1925 in Seattle W...,Sharon Tanagi Aburano Interview II ddrdensho10...,April 3 2008 Seattle Washington March 25 2008 ...,ddrdensho1000209 ddrdensho1000208
3,Toshiko Aiboshi,Nisei female Born July 8 1928 in Boyle Heights...,Toshiko Aiboshi Interview ddrmanz1112,January 20 2011 Culver City California,ddrmanz1112
4,Douglas L. Aihara,Sansei male Born March 15 1950 in Torrance Cal...,Douglas L Aihara Interview ddrdensho1000522,9Nov22 Los Angeles California,ddrdensho1000522


In [11]:
#(2) Remove numbers.
import pandas as pd
import re

input_file = "/content/Step_1_List_of_narrators.csv"
output_file = "/content/Step_2_List_of_narrators.csv"
def remove_numbers(text):
    if isinstance(text, str):
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
    return text
df = pd.read_csv(input_file, dtype=str)
df = df.applymap(remove_numbers)
df.to_csv(output_file, index=False)
df_cleaned = pd.read_csv(output_file)
df_cleaned.head()


<ipython-input-11-1fd3b17334aa>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_numbers)


,Narrator Name,Bio,Interview Title,Date & Location,Densho ID
0,Kay Aiko Abe,Nisei female Born May in Selleck Washington Sp...,Kay Aiko Abe Interview ddrdensho,December Seattle Washington,ddrdensho
1,Art Abe,Nisei male Born June in Seattle Washington Gre...,Art Abe Interview ddrdensho,January Seattle Washington,ddrdensho
2,Sharon Tanagi Aburano,Nisei female Born October in Seattle Washingto...,Sharon Tanagi Aburano Interview II ddrdensho S...,April Seattle Washington March Seattle Washington,ddrdensho ddrdensho
3,Toshiko Aiboshi,Nisei female Born July in Boyle Heights Califo...,Toshiko Aiboshi Interview ddrmanz,January Culver City California,ddrmanz
4,Douglas L. Aihara,Sansei male Born March in Torrance California ...,Douglas L Aihara Interview ddrdensho,Nov Los Angeles California,ddrdensho


In [12]:
# (3) Remove stopwords by using the stopwords list.
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
input_file = "/content/Step_2_List_of_narrators.csv"
output_file = "/content/Step_3_List_of_narrators.csv"
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    if isinstance(text, str):
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)
    return text
df = pd.read_csv(input_file, dtype=str)
df = df.select_dtypes(include=['object']).applymap(remove_stopwords)
df.to_csv(output_file, index=False)
df_cleaned = pd.read_csv(output_file)
df_cleaned.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-12-43fe60551573>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.select_dtypes(include=['object']).applymap(remove_stopwords)


,Narrator Name,Bio,Interview Title,Date & Location,Densho ID
0,Kay Aiko Abe,Nisei female Born May Selleck Washington Spent...,Kay Aiko Abe Interview ddrdensho,December Seattle Washington,ddrdensho
1,Art Abe,Nisei male Born June Seattle Washington Grew a...,Art Abe Interview ddrdensho,January Seattle Washington,ddrdensho
2,Sharon Tanagi Aburano,Nisei female Born October Seattle Washington F...,Sharon Tanagi Aburano Interview II ddrdensho S...,April Seattle Washington March Seattle Washington,ddrdensho ddrdensho
3,Toshiko Aiboshi,Nisei female Born July Boyle Heights Californi...,Toshiko Aiboshi Interview ddrmanz,January Culver City California,ddrmanz
4,Douglas L. Aihara,Sansei male Born March Torrance California Gre...,Douglas L Aihara Interview ddrdensho,Nov Los Angeles California,ddrdensho


In [13]:
#(4) Lowercase all texts
import pandas as pd

input_file = "/content/Step_3_List_of_narrators.csv"
output_file = "/content/Step_4_List_of_narrators.csv"
df = pd.read_csv(input_file, dtype=str).apply(lambda x: x.str.lower() if x.dtype == "object" else x)
df.to_csv(output_file, index=False)
df_cleaned = pd.read_csv(output_file)
df_cleaned.head()


,Narrator Name,Bio,Interview Title,Date & Location,Densho ID
0,kay aiko abe,nisei female born may selleck washington spent...,kay aiko abe interview ddrdensho,december seattle washington,ddrdensho
1,art abe,nisei male born june seattle washington grew a...,art abe interview ddrdensho,january seattle washington,ddrdensho
2,sharon tanagi aburano,nisei female born october seattle washington f...,sharon tanagi aburano interview ii ddrdensho s...,april seattle washington march seattle washington,ddrdensho ddrdensho
3,toshiko aiboshi,nisei female born july boyle heights californi...,toshiko aiboshi interview ddrmanz,january culver city california,ddrmanz
4,douglas l. aihara,sansei male born march torrance california gre...,douglas l aihara interview ddrdensho,nov los angeles california,ddrdensho


In [14]:
#(5) Stemming
import pandas as pd
import nltk
from nltk.stem import PorterStemmer

nltk.download('punkt', force=True)

input_file = "/content/Step_4_List_of_narrators.csv"
output_file = "/content/Step_5_List_of_narrators.csv"
st = PorterStemmer()
df = pd.read_csv(input_file, dtype=str).fillna('')
df['Bio'] = df['Bio'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df.to_csv(output_file, index=False)
df_cleaned = pd.read_csv(output_file)
df_cleaned.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Narrator Name,Bio,Interview Title,Date & Location,Densho ID
0,kay aiko abe,nisei femal born may selleck washington spent ...,kay aiko abe interview ddrdensho,december seattle washington,ddrdensho
1,art abe,nisei male born june seattl washington grew ar...,art abe interview ddrdensho,january seattle washington,ddrdensho
2,sharon tanagi aburano,nisei femal born octob seattl washington famil...,sharon tanagi aburano interview ii ddrdensho s...,april seattle washington march seattle washington,ddrdensho ddrdensho
3,toshiko aiboshi,nisei femal born juli boyl height california e...,toshiko aiboshi interview ddrmanz,january culver city california,ddrmanz
4,douglas l. aihara,sansei male born march torranc california grew...,douglas l aihara interview ddrdensho,nov los angeles california,ddrdensho


In [16]:
!pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.3 MB/s eta 0:00:00


In [17]:
# (6) Lemmatization.
import pandas as pd
import spacy
from spellchecker import SpellChecker

nlp = spacy.load("en_core_web_sm")
spell = SpellChecker()

file_path = "/content/Step_5_List_of_narrators.csv"
output_file = "/content/Step_6_List_of_narrators.csv"

df = pd.read_csv(file_path, dtype=str).fillna('')

def correct_and_lemmatize(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""

    words = text.split()
    corrected_words = [spell.correction(word) if spell.correction(word) else word for word in words]
    corrected_text = " ".join(corrected_words)

    doc = nlp(corrected_text)
    return " ".join([token.lemma_ for token in doc])

df['Bio'] = df['Bio'].apply(correct_and_lemmatize)

df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")


Data saved to /content/Step_6_List_of_narrators.csv


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [18]:
#(1) Parts of Speech (POS) Tagging:
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm")

file_path = "/content/Step_6_List_of_narrators.csv"

df = pd.read_csv(file_path, dtype=str).fillna('')

def pos_analysis(text):
    if not isinstance(text, str) or text.strip() == "":
        return {'N': 0, 'V': 0, 'Adj': 0, 'Adv': 0}
    doc = nlp(text)
    pos_counts = {'N': 0, 'V': 0, 'Adj': 0, 'Adv': 0}

    for token in doc:
        if token.pos_ == "NOUN":
            pos_counts['N'] += 1
        elif token.pos_ == "VERB":
            pos_counts['V'] += 1
        elif token.pos_ == "ADJ":
            pos_counts['Adj'] += 1
        elif token.pos_ == "ADV":
            pos_counts['Adv'] += 1
    return pos_counts
df['POS_Counts'] = df['Bio'].apply(pos_analysis)
pos_df = pd.DataFrame(df['POS_Counts'].tolist())

def save_results(dataframe, filename):
    output_path = f"/content/{filename}"
    dataframe.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")
save_results(pos_df, "Pos_analysis_results.csv")
total_counts = pd.DataFrame([pos_df.sum()])
save_results(total_counts, "Total_pos_counts.csv")


Results saved to /content/Pos_analysis_results.csv
Results saved to /content/Total_pos_counts.csv


In [20]:
#Constituency Parsing and Dependency Parsing
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")
file_path = "/content/Step_6_List_of_narrators.csv"
df = pd.read_csv(file_path, dtype=str).fillna('')

if "Bio" in df.columns and not df.empty:
    bio_text = df.loc[0, 'Bio']
else:
    raise ValueError("Error: 'Bio' column not found or dataset is empty.")

def parse_dependencies(text):
    doc = nlp(text)
    return [(token.text, token.dep_, token.head.text) for token in doc]

print("\nDependency Parsing Tree for First Narator:")
dependency_tree = parse_dependencies(bio_text)

for word, relation, head in dependency_tree:
    print(f"{word} - {relation}- {head}")



Dependency Parsing Tree for First Narator:
nisei - amod- bear
female - amod- bear
bear - nsubj- select
may - aux- select
select - ccomp- spend
washington - dobj- select
spend - ROOT- spend
much - amod- beaverton
childhood - compound- beaverton
beaverton - dobj- spend
oregon - compound- father
father - appos- beaverton
own - xcomp- spend
farm - compound- influence
influence - dobj- own
early - amod- age
age - compound- parent
parent - nsubj- cover
cover - conj- spend
christian - amod- war
world - compound- war
war - dobj- cover
I - nsubj- remove
remove - relcl- war
portland - nmod- center
assemble - compound- center
center - compound- oregon
oregon - compound- work
minidoka - compound- work
concent - compound- camp
camp - compound- work
idaho - compound- war
war - compound- work
work - nsubj- establish
establish - ccomp- remove
success - compound- feed
volant - amod- program
program - compound- feed
feed - dobj- establish
homeless - compound- seattle
seattle - appos- feed
washington - 

In [21]:
#(3) Name Entity Recognition:
import pandas as pd
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

file_path = "/content/Step_6_List_of_narrators.csv"
df = pd.read_csv(file_path, dtype=str).fillna('')

columns_to_process = ["Narrator Name", "Bio", "Interview Title", "Date & Location"]
entity_counts = Counter()
unique_entities = set()

for column in columns_to_process:
    if column in df.columns:
        for text in df[column].dropna():
            doc = nlp(text)
            for ent in doc.ents:
                unique_entities.add((ent.text, ent.label_))

for _, label in unique_entities:
    entity_counts[label] += 1

print("\nName Entity Counts:")
for entity, count in entity_counts.items():
    print(f"{entity}: {count}")



Name Entity Counts:
PERSON: 1381
ORG: 614
GPE: 210
PRODUCT: 10
LOC: 27
NORP: 55
DATE: 119
CARDINAL: 31
QUANTITY: 3
FAC: 26
LANGUAGE: 3
EVENT: 14
MONEY: 2
LAW: 2
ORDINAL: 7
TIME: 3


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

Prompt:Give me a code using python to scrape data such as Product name, Link, description, concurrent page number and format it in a csv file

In [3]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import csv
import requests
import time
import random

# Function for exponential backoff
def sleep_with_backoff(attempt):
    sleep_time = min(2 ** attempt + random.uniform(0, 1), 10)  # Exponential backoff up to 10 sec
    print(f"Retrying in {sleep_time:.2f} seconds...")
    time.sleep(sleep_time)

# Initialize session
session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0'})

# Function to extract action links from a specific page of the GitHub Marketplace
def extract_action_links(page_number):
    main_url = f"https://github.com/marketplace?type=actions&page={page_number}"
    action_links = []
    print(f"Processing Page {page_number}")

    for attempt in range(5):  # Retry up to 5 times
        try:
            req = Request(main_url, headers={'User-Agent': 'Mozilla/5.0'})
            page = urlopen(req)
            soup = BeautifulSoup(page, 'html.parser')

            for action in soup.find_all("div", attrs={'data-testid': 'non-featured-item'}):
                action_link_tag = action.find("a", href=True)
                action_name = action_link_tag.text.strip() if action_link_tag else "N/A"
                action_link = f"https://github.com{action_link_tag['href']}" if action_link_tag else "N/A"

                action_links.append({
                    'Page Number': page_number,
                    'Action Name': action_name,
                    'Link': action_link,
                    'Description': "N/A"
                })

            print(f"✅ Extracted {len(action_links)} action links.")
            break  # Exit retry loop on success
        except requests.exceptions.RequestException as e:
            print(f"Error occurred while processing page {page_number}: {e}")
            sleep_with_backoff(attempt)

    return action_links

# Function to extract descriptions from each action's page
def extract_description(action_links):
    for action in action_links:
        url = action['Link']

        for attempt in range(5):  # Retry up to 5 times
            try:
                response = session.get(url, timeout=10)

                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'lxml')
                    div_tag = soup.find('div', attrs={'data-testid': 'about'})

                    if div_tag:
                        span_tag = div_tag.find('span')
                        if span_tag:
                            action['Description'] = span_tag.get_text().strip()

                break  # Exit retry loop on success
            except requests.exceptions.RequestException as e:
                print(f"Error fetching {url}: {e}")
                sleep_with_backoff(attempt)

    return action_links

# Function to save data to a CSV file
def save_to_csv(data, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['Page Number', 'Action Name', 'Link', 'Description'])
        writer.writeheader()
        writer.writerows(data)

# Main execution
if __name__ == "__main__":
    all_action_links = []

    # Step 1: Extract action links from pages 1 to 60
    for page_number in range(1, 60):
        action_links = extract_action_links(page_number)
        all_action_links.extend(action_links)

        # Adding a delay between requests to avoid hitting rate limits
        time.sleep(5)  # Increased to 5 seconds

    # Step 2: Extract descriptions for each action
    updated_action_links = extract_description(all_action_links)

    # Step 3: Save final data to a CSV file
    save_to_csv(updated_action_links, "github_actions_final_data.csv")

    print("✅ Data extraction completed and saved to 'github_actions_final_data.csv'.")


Processing Page 1
✅ Extracted 20 action links.
Processing Page 2
✅ Extracted 20 action links.
Processing Page 3
✅ Extracted 20 action links.
Processing Page 4
✅ Extracted 20 action links.
Processing Page 5
✅ Extracted 20 action links.
Processing Page 6
✅ Extracted 20 action links.
Processing Page 7
✅ Extracted 20 action links.
Processing Page 8
✅ Extracted 0 action links.
Processing Page 9
✅ Extracted 20 action links.
Processing Page 10
✅ Extracted 0 action links.
Processing Page 11
✅ Extracted 20 action links.
Processing Page 12
✅ Extracted 20 action links.
Processing Page 13
✅ Extracted 20 action links.
Processing Page 14
✅ Extracted 20 action links.
Processing Page 15
✅ Extracted 20 action links.
Processing Page 16
✅ Extracted 0 action links.
Processing Page 17
✅ Extracted 20 action links.
Processing Page 18
✅ Extracted 20 action links.
Processing Page 19
✅ Extracted 20 action links.
Processing Page 20
✅ Extracted 20 action links.
Processing Page 21
✅ Extracted 20 action links.
Proc

Prompt: Give me the python code to clean the extraced file by utilising the necessary libraries for it

In [8]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from textblob import TextBlob  # Import TextBlob for spell correction

# Download NLTK resources (run this once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the CSV file (update the path)
file_path = '/content/github_actions_final_data.csv'  # Update with actual file path
df = pd.read_csv(file_path)

# Function to preprocess text (cleaning, tokenizing, stopword removal, lemmatization, spell correction)
def preprocess_text(text):
    if pd.isna(text):
        return "No description available"

    # Remove special characters and HTML tags
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces
    text = text.lower()  # Convert to lowercase

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Spell Correction using TextBlob (can be slow for large datasets)
    corrected_tokens = [str(TextBlob(word).correct()) for word in tokens]

    # Rejoin tokens into a string
    return ' '.join(corrected_tokens)

# Function to perform data quality checks (handle missing values and duplicates)
def data_quality_checks(data):
    # Remove duplicates
    data = data.drop_duplicates()

    # Handle missing values (fill with placeholders)
    data['Action Name'] = data['Action Name'].fillna('Unknown')
    data['Description'] = data['Description'].fillna('No description available')

    return data

# Step 1: Preprocess the 'Action Name' and 'Description' columns
df['Action Name'] = df['Action Name'].apply(preprocess_text)
df['Description'] = df['Description'].apply(preprocess_text)

# Step 2: Perform Data Quality checks
final_data = data_quality_checks(df)

# Step 3: Save the processed data to a new CSV file
output_file_path = '/content/github_actions_processed_data.csv'  # Update with actual path
final_data.to_csv(output_file_path, index=False)

# Print the path to the new file for reference
print(f"✅ Processed data saved to {output_file_path}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ Processed data saved to /content/github_actions_processed_data.csv


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


Prompt:Give me a step by step procedure to web scrape the tweets by using the tweepy based on the API Key , and other information submitted.

In [1]:
!pip install tweepy pandas


In [2]:
import tweepy
import pandas as pd


In [3]:
# Twitter API credentials
api_key = "KRLtqQXjJajRmZ0sWoQItprmB"
api_key_secret = "WicaRTeR8R8cEmUFYOsTcaK4kpMIMeyBtdGkMtUiqfwORKaV9q"
access_token = "1523595813980622848-VhWF1LHPp3fv92NSZOaUXeNmfZqIP2"
access_token_secret = "kuuenAOWvtjo9jWthXQ6ADzabndtnM6HlAYzZSpW2I3EC"

# Authenticate with Twitter API
auth = tweepy.OAuth1UserHandler(api_key, api_key_secret, access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [5]:
query = "(#MachineLearning OR #ArtificialIntelligence OR #AI OR #ML) -is:retweet"

# Use Twitter API v2
client = tweepy.Client(bearer_token="AAAAAAAAAAAAAAAAAAAAAPZjzQEAAAAAXrMiXZsyuqaThW1cSqs2mKUM6Fw%3DGQRfUw39znCTMPdwcsNbX6yVyKMP03v8ddpEDykyWWDHTV42hp")

tweets = client.search_recent_tweets(query=query,
                                     tweet_fields=["created_at", "text", "author_id"],
                                     max_results=100)

# Store the tweets
tweet_data = []

for tweet in tweets.data:
    tweet_data.append({
        'tweet_id': tweet.id,
        'created_at': tweet.created_at,
        'user': tweet.author_id,  # API v2 does not return screen_name directly
        'text': tweet.text
    })


In [7]:
print(len(tweet_data))  # Check how many tweets were stored
print(tweet_data[:5])   # Print first 05 tweets for verification


100
[{'tweet_id': 1891722675149533520, 'created_at': datetime.datetime(2025, 2, 18, 5, 33, 26, tzinfo=datetime.timezone.utc), 'user': 1755770718623821824, 'text': '#AI,#ammo @Ammo_AI'}, {'tweet_id': 1891722647362236860, 'created_at': datetime.datetime(2025, 2, 18, 5, 33, 20, tzinfo=datetime.timezone.utc), 'user': 3302977957, 'text': '@JirinaSalova\n@VeraNejedla\n@MikaNetoka\n\n@Ammo_AI #AI #ammo'}, {'tweet_id': 1891722632308949139, 'created_at': datetime.datetime(2025, 2, 18, 5, 33, 16, tzinfo=datetime.timezone.utc), 'user': 1867667895003516928, 'text': '@misteriouspavao Excited for Apple glasses - the future is here! 🤓🍎🚀 #AR #VR #AI #Apple #CES2023'}, {'tweet_id': 1891722630056550642, 'created_at': datetime.datetime(2025, 2, 18, 5, 33, 16, tzinfo=datetime.timezone.utc), 'user': 1572634346217172993, 'text': "@robbystarbuck Merit *and* a diverse talent pool?  That's how PublicAI wins.  Checkmate. 😉 #AI #Data @PublicAIData\n @PublicAI_"}, {'tweet_id': 1891722626671731014, 'created_at': d

In [8]:
# Convert to DataFrame
df = pd.DataFrame(tweet_data)

# Save as CSV file
df.to_csv("tweets.csv", index=False, encoding="utf-8")

print("Tweets saved successfully to tweets.csv")



Tweets saved successfully to tweets.csv


In [15]:
df_check = pd.read_csv("tweets.csv")
print(df_check.head())

              tweet_id                 created_at                 user  \
0  1891722675149533520  2025-02-18 05:33:26+00:00  1755770718623821824   
1  1891722647362236860  2025-02-18 05:33:20+00:00           3302977957   
2  1891722632308949139  2025-02-18 05:33:16+00:00  1867667895003516928   
3  1891722630056550642  2025-02-18 05:33:16+00:00  1572634346217172993   
4  1891722626671731014  2025-02-18 05:33:15+00:00  1656610048628645888   

                                                text  
0                                 #AI,#ammo @Ammo_AI  
1  @JirinaSalova\n@VeraNejedla\n@MikaNetoka\n\n@A...  
2  @misteriouspavao Excited for Apple glasses - t...  
3  @robbystarbuck Merit *and* a diverse talent po...  
4  @MrJohnFKennedy @bankofireland Irish manufactu...  


Prompt: Get me the code by deep cleaning the file in python by implementing all the pre processing details needed

In [21]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob  # For spell checking
import emoji  # To remove emojis
import unidecode  # To normalize accented characters


# Download required NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")

# Load the scraped tweets
df = pd.read_csv("/content/tweets.csv")

# 1️⃣ Remove duplicates
df.drop_duplicates(subset="tweet_id", keep="first", inplace=True)

# 2️⃣ Drop missing values
df.dropna(inplace=True)

# 3️⃣ Remove URLs, special characters, emojis, and extra spaces
def clean_text(text):
    text = unidecode.unidecode(text)  # Normalize accented characters
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"@[A-Za-z0-9_]+", "", text)  # Remove mentions (@user)
    text = re.sub(r"#(\w+)", r"\1", text)  # Remove # but keep the word
    text = re.sub(r"[^\w\s]", "", text)  # Remove special characters & punctuation
    text = emoji.replace_emoji(text, replace="")  # Remove emojis
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text.lower()  # Convert to lowercase

df["text"] = df["text"].apply(clean_text)

# 4️⃣ Remove stopwords & perform lemmatization
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

df["text"] = df["text"].apply(preprocess_text)

# 5️⃣ Spell Check & Auto-correct text
def correct_spelling(text):
    return str(TextBlob(text).correct())  # Uses TextBlob to fix misspellings

df["text"] = df["text"].apply(correct_spelling)

# 6️⃣ Convert 'created_at' to datetime format
df["created_at"] = pd.to_datetime(df["created_at"])

# 7️⃣ Final quality check
print(" Final dataset shape:", df.shape)
print(df.head())

# Save cleaned data to a new CSV file
cleaned_filename = "final_cleaned_tweets.csv"
df.to_csv(cleaned_filename, index=False, encoding="utf-8")

print(f"🎉 Deep cleaning complete! Cleaned data saved as '{cleaned_filename}'.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 Final dataset shape: (100, 4)
              tweet_id                created_at                 user  \
0  1891722675149533520 2025-02-18 05:33:26+00:00  1755770718623821824   
1  1891722647362236860 2025-02-18 05:33:20+00:00           3302977957   
2  1891722632308949139 2025-02-18 05:33:16+00:00  1867667895003516928   
3  1891722630056550642 2025-02-18 05:33:16+00:00  1572634346217172993   
4  1891722626671731014 2025-02-18 05:33:15+00:00  1656610048628645888   

                                                text  
0                                              alamo  
1                                            ai also  
2  excited apple glass future ar or ai apple ces2023  
3  merit diverse talent pool that publican win ch...  
4  irish manufacturing 15 export growth impressiv...  
🎉 Deep cleaning complete! Cleaned data saved as 'final_cleaned_tweets.csv'.


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog